# Merge all auctions into one

Combines several datasets and do some simple analyses.  


### User variables

In [1]:
import os
# setting path
os.chdir(r'..')

import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
SKIPSAVE = cfg['SKIPSAVE']

TAG_SINGLE = "nbconvert_instruction:remove_single_output"

if VERBOSE > 0:
    display(cfg)

{'settings_fn': '../code/assets/drz-auction-settings.ini',
 'DATE': '2022-10',
 'VERBOSE': 1,
 'OPBOD': False,
 'URL': 'http://verkoop.domeinenrz.nl/verkoop_bij_inschrijving_2022-0010',
 'EXTEND_URL': False,
 'CLOSEDDATA': True,
 'closed_data_fields': '*',
 'SKIPSAVE': False}

In [2]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
if OPBOD:
    AuctionDates = ['2019-11', '2019-12', 
                    '2020-01', '2020-02', '2020-03', '2021-09-09', '2021-10-10', '2021-11-11', '2021-12-12', 
                    '2022-01-01', '2022-02-02', '2022-04-04', '2022-05-05', '2022-06-06', '2022-08-08', '2022-09-09', '2022-10-10'
                   ]
else:    
    AuctionDates = ['2014-10', '2014-11', '2014-12', # old!
                    '2015-01', '2015-02', '2015-03', # old!
                    '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
                    '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                    '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                    '2020-01', '2020-02', '2020-03', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                    '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', 
                    '2021-06', '2021-06-16', '2021-07-07',  '2021-07-17', '2021-08-08', '2021-08-18', '2021-09-09', '2021-09-19', '2021-10-10', '2021-10-20', '2021-11-11', '2021-11-21', '2021-12-12', '2021-12-22',
                    '2022-01-01', '2022-01-21', '2022-02-02', '2022-02-22', '2022-03-03', '2022-03-23', '2022-04-04', '2022-04-24', '2022-05-05', '2022-05-25', '2022-06-06', 
                    '2022-07-07', '2022-07-27', '2022-08-08', '2022-08-28', '2022-09-09', '2022-09-29', '2022-10-10'
                   ]  
if DATE not in ['-'.join(d.split('-')[:2]) for d in AuctionDates]:
    raise ValueError(f'{DATE} not in list that will be concatenated. Add to list.')

### Import modules 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
plt.style.use('ggplot')

### Load all data

Auctions are saved monthly in a .pkl file

In [4]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    if OPBOD:
        fn = '../data/rdw-data-{:s}-opbod.pkl'.format(
            AuctionDate
        )

    else:    
        fn = '../data/rdw-data-{:s}.pkl'.format(
            AuctionDate
        )

    #if VERBOSE > 0: print(fn)
    df = pd.read_pickle(fn)
    data[AuctionDate] = df
    
    
if VERBOSE > 0: 
    print('fix indices')

# matrix of auction date
auct_month = np.array([[int(k.split('-')[0]), int(k.split('-')[1])] for k in data.keys()])
auct_month = np.unique(auct_month, axis=0)

# As of 2021-07 there is a systematic index change
#    this is cause by multiple auctions per month
for m in auct_month[(auct_month[:,0] == 2021) & (auct_month[:,1] >= 7) | (auct_month[:,0] >= 2022)]:
    
    # Auctions per month
    for c in range(2): 

        # multiplier
        #   2021: 10
        #   2022: 20
        if m[0] == 2021:
            multiplier = 10
        elif m[0] == 2022:
            multiplier = 20
        else:
            raise NotImplementedError
        
        # data key
        #    yyyy-mm-xx xx: mm + multiplier
        k = f'{m[0]}-{m[1]:02.0f}-{m[1]+(c*multiplier):02.0f}'
        if k not in data.keys():
            # skip if not exist
            continue
        
        # translation
        #    yyyy-mm-llllxx: llll: lot number
        pat = '{0}-' + f'{m[1]:02.0f}' + '-{2}{1}'
        data[k].index = data[k].index.map(lambda x: pat.format(*x.split('-')))
        
        if VERBOSE > 0:
            print(f'{k} > {data[k].index[0]} .. {data[k].index[-1]}')



load data
fix indices
2021-07-07 > 2021-07-180107 .. 2021-07-970007
2021-07-17 > 2021-07-180017 .. 2021-07-809417
2021-08-08 > 2021-08-180008 .. 2021-08-960308
2021-08-18 > 2021-08-190018 .. 2021-08-960018
2021-09-09 > 2021-09-180009 .. 2021-09-904509
2021-09-19 > 2021-09-190019 .. 2021-09-808919
2021-10-10 > 2021-10-180010 .. 2021-10-970210
2021-10-20 > 2021-10-190020 .. 2021-10-970120
2021-11-11 > 2021-11-180011 .. 2021-11-708711
2021-11-21 > 2021-11-190121 .. 2021-11-970121
2021-12-12 > 2021-12-180012 .. 2021-12-715712
2021-12-22 > 2021-12-190022 .. 2021-12-807722
2022-01-01 > 2022-01-180001 .. 2022-01-960001
2022-01-21 > 2022-01-190021 .. 2022-01-805621
2022-02-02 > 2022-02-180002 .. 2022-02-900002
2022-02-22 > 2022-02-190022 .. 2022-02-970222
2022-03-03 > 2022-03-180003 .. 2022-03-710903
2022-03-23 > 2022-03-190023 .. 2022-03-900023
2022-04-04 > 2022-04-180004 .. 2022-04-707404
2022-04-24 > 2022-04-190024 .. 2022-04-970124
2022-05-05 > 2022-05-180005 .. 2022-05-970005
2022-05-25 >

In [5]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    if OPBOD:
        fn = '../data/rdw-data-{:s}-opbod.pkl'.format(
            AuctionDate
        )

    else:    
        fn = '../data/rdw-data-{:s}.pkl'.format(
            AuctionDate
        )

    if VERBOSE > 0: print(fn)
    df = pd.read_pickle(fn)
    data[AuctionDate] = df

    
    
# When there are two auctions in a single month
if not OPBOD:
    # HACK ALERT!!
    data['2021-06-16'].index = data['2021-06-16'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    data['2021-06'].index = data['2021-06'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    
    # matrix of auction date
    auct_month = np.array([[int(k.split('-')[0]), int(k.split('-')[1])] for k in data.keys()])
    auct_month = np.unique(auct_month, axis=0)

    # 2021-07 there is a systematic index change
    for m in auct_month[(auct_month[:,0] == 2021) & (auct_month[:,1] >= 7) | (auct_month[:,0] >= 2022)]:
        for c in range(2):

            # multiplier
            #   2021: 10
            #   2022: 20
            if m[0] == 2021:
                multiplier = 10
            elif m[0] == 2022:
                multiplier = 20
            else:
                raise NotImplementedError

            # data key
            k = f'{m[0]}-{m[1]:02.0f}-{m[1]+(c*multiplier):02.0f}'
            if k not in data.keys():
                # skip if not exist
                continue

            # translation
            pat = '{0}-' + f'{m[1]:02.0f}' + '-{2}{1}'
            data[k].index = data[k].index.map(lambda x: pat.format(*x.split('-')))

            if VERBOSE > 0:
                print(f'{k} > {data[k].index[0]} .. {data[k].index[-1]}')
if VERBOSE > 0:
    display({'text/plain': '\nlast rows of last file'}, raw=True, metadata={"tags":(TAG_SINGLE, )})
    display(data[AuctionDates[-1]].tail(), metadata={"tags":(TAG_SINGLE, )})

load data
../data/rdw-data-2014-10.pkl
../data/rdw-data-2014-11.pkl
../data/rdw-data-2014-12.pkl
../data/rdw-data-2015-01.pkl
../data/rdw-data-2015-02.pkl
../data/rdw-data-2015-03.pkl
../data/rdw-data-2017-03.pkl
../data/rdw-data-2017-04.pkl
../data/rdw-data-2017-05.pkl
../data/rdw-data-2017-06.pkl
../data/rdw-data-2017-07.pkl
../data/rdw-data-2017-08.pkl
../data/rdw-data-2017-09.pkl
../data/rdw-data-2017-10.pkl
../data/rdw-data-2017-11.pkl
../data/rdw-data-2017-12.pkl
../data/rdw-data-2018-01.pkl
../data/rdw-data-2018-02.pkl
../data/rdw-data-2018-03.pkl
../data/rdw-data-2018-04.pkl
../data/rdw-data-2018-05.pkl
../data/rdw-data-2018-06.pkl
../data/rdw-data-2018-07.pkl
../data/rdw-data-2018-08.pkl
../data/rdw-data-2018-09.pkl
../data/rdw-data-2018-10.pkl
../data/rdw-data-2018-11.pkl
../data/rdw-data-2018-12.pkl
../data/rdw-data-2019-01.pkl
../data/rdw-data-2019-02.pkl
../data/rdw-data-2019-03.pkl
../data/rdw-data-2019-04.pkl
../data/rdw-data-2019-05.pkl
../data/rdw-data-2019-06.pkl
../d

### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [6]:
# Change type of early auctions for images
if VERBOSE > 0:
    print('Fields with images look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first row'])
    for k in data:
        im_cols = data[k].columns[
            data[k].columns.str.lower().isin(['image', 'images'])
        ]
        example = data[k][im_cols[0]].values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = im_cols[0]
        if isinstance(example, str):
            df_.loc[k, 'first row'] = example
        else:
            df_.loc[k, 'first row'] = len(example)
    with pd.option_context('display.max_row', 999):
        display(df_)

if OPBOD:
    # reformatting did not happen in IRS auctions
    pass
else:
    
    # convert string representation of list to real list
    # "['image1', 'image2']"
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        df_ = data[k].copy() # avoid performance warning
        df_['Images'] = df_['Image'].apply(eval)
        data[k] = df_.drop(columns=['Image'])

    # convert string representation of list without quotes to real list
    # "[image1, image2]"
    for k in ['2017-11',
              '2017-12',
              '2018-01',
              '2018-02',
              '2018-03',
              '2018-04']:
        data[k]['Images'] = data[k]['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
    

# Other inconsistencies
if OPBOD:
    pass
else:
    # Other column name that changed in November 2017
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k] = data[k].rename(columns={
            'draw':'Draw',
            'nr':'LotNr',
            'raw':'Raw_text',
            'misc':'SupInfo',
            'footnote':'Note',
            'jfq':'jfc',
        })

Fields with images look like this:



,column,type,first row
2014-10,Images,<class 'list'>,0
2014-11,Images,<class 'list'>,0
2014-12,Images,<class 'list'>,0
2015-01,Images,<class 'list'>,0
2015-02,Images,<class 'list'>,0
2015-03,Images,<class 'list'>,0
2017-03,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14883593...
2017-04,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14906980...
2017-05,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14931056...
2017-06,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14962307...


# Merge all auctions into one data frame

In [7]:
all_data = pd.concat(data)
df = all_data.droplevel(0)

In [8]:
if VERBOSE > 0:
    # Show columns added
    col_exist = pd.DataFrame.from_dict({k:all_data.columns.isin(d.columns) for k,d in data.items()}, orient='index', columns=all_data.columns)
    # deprecated: col_filled = all_data.isna().all(level=0, axis=0) == False
    col_filled = all_data.notna().groupby(level=0).any()

    idx = col_exist.apply(lambda x:np.where(x)[0])

    first_occurence = col_exist.index[idx.apply(lambda x: x[0])]
    last_occurence = col_exist.index[idx.apply(lambda x: x[-1])]
    nr_occurence = idx.apply(len)
    pct_occurence = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])))

    idx = col_filled.apply(lambda x:np.where(x)[0])
    last_filled = col_filled.index[idx.apply(lambda x: x[-1] if len(x)>0 else col_exist.shape[0]-1)]
    nr_filled = idx.apply(len)
    pct_filled = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])) if len(x)>0 else 0)


    df_ = pd.DataFrame(zip(first_occurence, last_occurence, nr_occurence, last_filled, nr_filled, pct_occurence, pct_filled), 
                       index=col_exist.columns, 
                       columns=['first', 'last', 'n', 'last_filled', 'n_not_empty', 'pct_since_first', 'pct_filled'])



    for _by_rdw, df__ in df_.groupby(df_.index.str.startswith('rdw')):
        for g,df___ in df__.groupby('first'):
            if g == col_exist.index[0]:
                continue
            print(f'Fields added in {g}', end= ' ')
            if _by_rdw:
                print('by rdw api')
            else:
                print('through regex parsing')
            with pd.option_context("display.max_rows", 999):
                display(df___.sort_index().sort_values(by='last_filled', ascending=False))

Fields added in 2017-11 through regex parsing


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
used_parts,2017-11,2017-11,1,2017-11,1,1.369863,1.369863


Fields added in 2018-01 through regex parsing


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
import22_btw21,2018-01,2018-07,2,2018-07,2,2.816901,2.816901
import27_btw21,2018-01,2018-07,2,2018-07,2,2.816901,2.816901


Fields added in 2018-04 through regex parsing


...

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_brandstof_opgegeven_maximum_snelheid_2,2022-09-09,2022-09-09,1,2022-10-10,0,33.333333,0.000000
rdw_brandstof_opgegeven_maximum_snelheid_1,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_ovi_breedte_min_max,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_ovi_lengte_min_max,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_ovi_massa_rijklaar_min_max,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_ovi_opgegeven_max_snelheid,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_ovi_technische_maximum_massa_voertuig_min_max,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_ovi_voertuigomschrijving,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_ovi_wielbasis_min_max,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333
rdw_subcategorie_nederland,2022-09-09,2022-09-09,1,2022-09-09,1,33.333333,33.333333


Fields added in 2022-09-29 by rdw api


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_merk_uitvoering_toegestaan_merk_registratie_datum_1,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_40,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_32,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_33,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_34,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_35,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_36,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_37,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_38,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_39,2022-09-29,2022-10-10,2,2022-10-10,2,100.0,100.0


Fields added in 2022-10-10 by rdw api


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_merk_uitvoering_toegestaan_merk_registratie_datum_100,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_116,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_128,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_127,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_126,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_125,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_124,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_123,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_122,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0
rdw_merk_uitvoering_toegestaan_merkcode_121,2022-10-10,2022-10-10,1,2022-10-10,1,100.0,100.0


- - - -
# Continue with cars only

In [9]:
# select data
car=df[df.LotType == 'Personenauto'].copy()

if VERBOSE > 0:
    # print lots that are cars per auction
    df_ = pd.DataFrame(columns=['year', 'month', 'lot'])
    df_.year = pd.Series(car.index).apply(lambda x: int(x.split('-')[0]))
    df_.month = pd.Series(car.index).apply(lambda x: int(x.split('-')[1]))
    df_.lot = pd.Series(car.index).apply(lambda x: int(x.split('-')[2]))
    with pd.option_context('display.max_rows', 999):
        display(df_.groupby(['year', 'month']).lot.agg(['min', 'max', 'count']).sort_index())
    

display(car.tail(), metadata={"tags":(TAG_SINGLE, )})

min     max  count
year month                       
2014 10       2000    7312    136
     11       2400    8198    133
     12       2000    7306    147
2015 1        2402    8318    134
     2        2000    7318    138
     3        2400    8312    157
2017 3        2000    8318    149
     4        2000    7309    133
     5        2200    8025     75
     6        2000    7311    140
     7        2200    8178    106
     8        2001    7304    143
     9        2200    8187    133
     10       2000    7171     95
     11       2200    8305    158
     12       2000    7308    136
2018 1        2200    8170    134
     2        2000    7184    138
     3        2200    8338    186
     4        2000    7336    126
     5        2200    8226    186
     6        2000    7315    166
     7        2200    8311    132
     8        2000    7328    152
     9        2200    8183     89
     10       2000    7319     87
     11       2200    8326    127
     12       2000    7361    133
2019 1        2201    8329    135
     2        2001    7264    176
     3        2200    8318    145
     4        2000    7314    170
     5        2200    8352    158
     6        2201    9158     98
     7        2200    7254    144
     8        2200    8172     96
     9        2200    9269    156
     10       2400    7200    109
     11       2200    8213    125
     12       2200    9061     75
2020 1        2200    7205    126
     2        2200    8238    129
     3        2200    9152     65
     6        2200    7269    160
     7        2200    9271    145
     8        2200    7286    187
     9        2200    8236    111
     10       2200    7266    129
     11       2200    8260    121
     12       2201    7306    168
2021 1        2200    8123    104
     2        2200    7156    129
     3        2200    8189    148
     4        2200    7220    173
     5        2200    8158    126
     6      220006  808316    191
     7      201217  900007    152
     8      260018  812918    143
     9      220009  808919    135
     10     260110  814920    181
     11     262411  805621     97
     12     260012  807722    171
2022 1      220021  805621    115
     2      240002  810122    193
     3      220003  807723    132
     4      240004  805724     88
     5      260025  805125    103
     6      700006  712506     79
     7      260007  982727    112
     8      260008  803228     81
     9      265029  803429    106
     10     700810  807010     30

# Save file

In [10]:
file_name = r'../data/cars-from-all-auctions.pkl'
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    car.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    


../data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [11]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [12]:
file_name = '../assets/example-table-of-cars.md'

example = car.tail(10).copy()

# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'

# convert to md
try:
    table_text = example.to_markdown()
except ImportError:
    print('Fallback')
    table_text = pandas_df_to_markdown_table(example)

# save
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    with open(file_name,'w') as file:
        file.write(table_text)

    print('A markdown table is available as\n\t{}'.format(file_name))
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    display(example, metadata={"tags":(TAG_SINGLE, )})
    


Fallback
Skip. ../assets/example-table-of-cars.md exists or saving is disabled in settings.


In [13]:
# idx = df.index.str.split('-', expand=True).droplevel(-1)
# is_last_auction = idx == idx[-1]
# subsel_reg = df.loc[is_last_auction, 'Reg'].drop_duplicates().values
# VERBOSE

In [14]:
# vc2

In [15]:
# show recurring lots (based on registration no.)

vc = df.Reg.value_counts()

# drop obvious
vc.drop(['onbekend', ''], inplace=True, errors='ignore')

# reduce output
if VERBOSE < 2:
    # only of last auction
    idx = df.index.str.split('-', expand=True).droplevel(-1)
    is_last_auction = idx == idx[-1]
    subsel_reg = df.loc[is_last_auction, 'Reg'].drop_duplicates().values
    vc = vc[vc.index.isin(subsel_reg)]
    
# show recurring
for r in vc[vc > 1].index:
    # title
    print(r, vc[r], end=': ')
    sel_reg = df.Reg==r
    print(' - '.join(list(df.loc[sel_reg, ['ItemBrand', 'ItemType', 'rdw_datum_eerste_afgifte_nederland']].drop_duplicates(ignore_index=True).iloc[0].astype(str))), end=' ')
    
    df_ = df.loc[sel_reg, list(df.columns.drop([
    'Raw_text', 'rdwinfo', 'rdw_merk_registratie_datum', 'rdw_merk_registratie_datum_dt', 'rdw_merkcode', 'Images', 'Source'
    ], errors='ignore'))].sort_values(by='Reg').T

    # convert index to nr of days since first
    ix = pd.Series(df_.columns.map(lambda x: pd.to_datetime('/'.join(x.split('-')[:2])))).diff().dt.days.copy()
    ix[0] = 0
    ix=ix.astype(int).astype(str)
    ix[0] = pd.to_datetime('/'.join(df_.columns[0].split('-')[:2])).strftime('%Y/%m')

    df_.columns=ix

    if (VERBOSE < 2) | (df_.shape[1] > 4):

        vc2 = df_.applymap(lambda x: str(x) if isinstance(x, list) else x).nunique(axis=1)
        print('')
        display(df_.loc[list(vc2[vc2>1].index),:])
    elif df_.shape[1] > 2:
        display(df_.loc[['Price'],:].T)
    else:
        print(*list(df_.columns))
        

FFK-37-N 3: CAPRI - cl 50 - 20201224.0 


,2021/12,31,273
Title,Kavel K2100121883,Kavel K2200011853,Kavel K2200101812
Price,0.0,0.0,369.0
N_images,7.0,7.0,8.0
lot_counter,1883.0,1853.0,1812.0
LotNr,K2100121883,K2200011853,K2200101812
OdoKM,4.214,4.214,5.723
no_rdw,False,False,True
rdw_datum_tenaamstelling,20211103.0,20211103.0,20220912.0
rdw_datum_tenaamstelling_dt,2021-11-03T00:00:00.000,2021-11-03T00:00:00.000,2022-09-12T00:00:00.000
rdw_TimeStamp_x,20211210,20220107,20221008


44-LR-ZB 3: MINI - cooper - nan 


,2022/07,31,61
Title,Kavel K2200077055,Kavel K2200288032,Kavel K2200107010
Price,3003.0,0.0,2111.0
lot_counter,7055.0,8032.0,7010.0
LotNr,K2200077055,K2200288032,K2200107010
ItemType,cooper,mini cooper; cooper,cooper
rdw_TimeStamp_x,20220709,20220901,20221008
rdw_api_carrosserie_uitvoering_klasse,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
rdw_api_carrosserie_uitvoering,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
rdw_TimeStamp_y,20220709,20220901,20221008
rdw_ovi_plaats_chassis_omschrijving,"Op rechts, rechter","Op rechts, rechter schroefveerkoker onder moto...","Op rechts, rechter schroefveerkoker onder moto..."


XS-502-J 2: VOLKSWAGEN - polo - 20190305.0 


...

,2022/08,61
Title,Kavel K2200288004,Kavel K2200107009
lot_counter,8004.0,7009.0
LotNr,K2200288004,K2200107009
rdw_TimeStamp_x,20220901,20221008
rdw_TimeStamp_y,20220901,20221008


1-XHR-45 2: MITSUBISHI - outlander - nan 


,2022/05,153
Title,Kavel K2200057078,Kavel K2200107046
Price,0.0,13815.0
lot_counter,7078.0,7046.0
LotNr,K2200057078,K2200107046
rdw_wam_verzekerd,Ja,Nee
rdw_TimeStamp_x,20220509,20221008
rdw_api_carrosserie_uitvoering_klasse,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
rdw_api_carrosserie_uitvoering,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
rdw_TimeStamp_y,20220509,20221008
rdw_ovi_w_a_verzekerd,Ja,Nee


4-KKP-49 2: BMW - 320i - 20130515.0 


,2022/03,214
Title,Kavel K2200238048,Kavel K2200107015
Price,0.0,2669.0
N_images,11.0,14.0
lot_counter,8048.0,7015.0
LotNr,K2200238048,K2200107015
rdw_TimeStamp_x,20220326,20221008
rdw_ovi_eigenaren,7 / 0,5 / 0
rdw_ovi_private_owners,7.0,5.0
rdw_ovi_owners,7.0,5.0
